<a href="https://colab.research.google.com/github/alighasemi78/persian_paraphraser/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning

## Installing the Required Libraries

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu
!apt install git-lfs

## Importing the Required Libraries

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np

## Login to Huggingface

In [ ]:
notebook_login()

## Loading the Dataset and Metric

In [ ]:
raw_datasets = load_dataset("alighasemi/fa-paraphrase")
metric = load_metric("sacrebleu")

In [ ]:
raw_datasets

In [ ]:
raw_datasets["train"][0]

In [ ]:
metric

In [ ]:
fake_preds = ["سلام خوبی", "او بازیکن خوبی است"]
fake_labels = [["سلام دوست من"], ["او بازیکن خوبی است"]]
metric.compute(predictions=fake_preds, references=fake_labels)

## Loading the Tokenizer

In [ ]:
model_checkpoint = "alighasemi/fa-t5-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("سلام، این یک جمله است!")

In [ ]:
tokenizer(["سلام، این یک جمله است!", "این یک جمله دیگر است."])

In [ ]:
tokenizer(text_target=["سلام، این یک جمله است!", "این یک جمله دیگر است."])

In [ ]:
prefix = "paraphrase: "

## Preprocess the Data

In [ ]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["sentence1"]]
    targets = [ex for ex in examples["sentence2"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(small_raw_datasets['train'][:2])

In [ ]:
tokenized_datasets = small_raw_datasets.map(preprocess_function, batched=True)

## Loading the Model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Creating the Training Arguments

In [ ]:
batch_size = 16
model_name = "fa-t5-paraphraser"
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Computing the Metrics

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Training

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()